In [4]:
    from stellar_base.keypair import Keypair
    from stellar_base.asset import Asset
    from stellar_base.operation import Payment
    from stellar_base.transaction import Transaction
    from stellar_base.transaction_envelope import TransactionEnvelope as Te
    from stellar_base.memo import TextMemo
    from stellar_base.horizon import horizon_testnet, horizon_livenet
    
    alice_seed = 'SAZJ3EDATROKTNNN4WZBZPRC34AN5WR43VEHAFKT5D66UEZTKDNKUHOK'
    bob_address = 'GBTQMUC6AUUD7LP6PCJGXFWIYK32LT24XPAGTMUMIEF5I3CANBBVAUA2'
    CNY_ISSUER = 'GDVDKQFP665JAO7A2LSHNLQIUNYNAAIGJ6FYJVMG4DT3YJQQJSRBLQDG'
    amount = '100'
    
    Alice = Keypair.from_seed(alice_seed)
    horizon = horizon_testnet() # horizon = horizon_livenet() for LIVENET
    
    asset = Asset('CNY', CNY_ISSUER) 
    # create op 
    op = Payment({
        # 'source' : Alice.address().decode(),
        'destination': bob_address,
        'asset': asset,
        'amount': amount
    })
    # create a memo
    msg = TextMemo('Buy yourself a beer !')
    
    sequence = horizon.account(Alice.address().decode('utf-8')).get('sequence')
    
    # construct Tx
    tx = Transaction(
        source=Alice.address().decode(),
        opts={
            'sequence': sequence,
            # 'timeBounds': [],
            'memo': msg,
            # 'fee': 100,
            'operations': [
                op,
            ],
        },
    )
    
    
    # build envelope
    envelope = Te(tx=tx, opts={"network_id": "TESTNET"}) # envelope = Te(tx=tx, opts={"network_id": "PUBLIC"}) for LIVENET
    # sign 
    envelope.sign(Alice)
    # submit
    xdr = envelope.xdr()
    response=horizon.submit(xdr)

In [6]:
response

{'detail': 'The transaction failed when submitted to the stellar network. The `extras.result_codes` field on this response contains further details.  Descriptions of each code can be found at: https://www.stellar.org/developers/learn/concepts/list-of-operations.html',
 'extras': {'envelope_xdr': 'AAAAAKz4N91L2xSNuJURDEON/mMoLk0UskIIyA+SRJluVG3NAAAAZAAVvagAAAAMAAAAAAAAAAEAAAAVQnV5IHlvdXJzZWxmIGEgYmVlciAhAAAAAAAAAQAAAAAAAAABAAAAAGcGUF4FKD+t/niSa5bIwrelz1y7wGmyjEEL1GxAaENQAAAAAUNOWQAAAAAA6jVAr/e6kDvg0uR2rgijcNABBk+LhNWG4Oe8JhBMohUAAAAAO5rKAAAAAAAAAAABblRtzQAAAEDH35yDgLNGzeFMWo9oCDNSCxWmbByA65JlHaafdkCheJ2GwSAxsi2AD24trnHg6uDc3sHcerCsObcT3+wwJIUG',
  'result_codes': {'operations': ['op_no_trust'], 'transaction': 'tx_failed'},
  'result_xdr': 'AAAAAAAAAGT/////AAAAAQAAAAAAAAAB////+gAAAAA='},
 'instance': 'horizon-testnet-001/6VNfUsVQkZ-3613742',
 'status': 400,
 'title': 'Transaction Failed',
 'type': 'https://stellar.org/horizon-errors/transaction_failed'}